# Интеллектуальный анализ текстов

In [6]:
import scipy.spatial
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

### Расстояние Левенштейна 

In [9]:
fuzz.ratio("Игра", "Играть")/100

0.8

In [10]:
fuzz.ratio("Играющий", "Играть")/100

0.57

In [12]:
fuzz.ratio("Обедать", "Играть")/100

0.46

### Косинусное растояние

In [18]:
sentences = [
    'привет друг',
    'привет привет',
    'как дела друг',
    'доброе утро',
    'что делаешь'
]
tokens = set(' '.join(sentences).split())
vectors = [[sentence.count(token) 
            for token in tokens]
           for sentence in sentences]
print([(i, word) for i, word in enumerate(tokens)])
vectors

[(0, 'утро'), (1, 'друг'), (2, 'как'), (3, 'доброе'), (4, 'делаешь'), (5, 'дела'), (6, 'что'), (7, 'привет')]


[[0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 2],
 [0, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 0]]

In [21]:
for i, sentence in enumerate(sentences):
    print(sentence)
    print(scipy.spatial.distance.cosine(vectors[0], vectors[i]))

привет друг
0.0
привет привет
0.29289321881345254
как дела друг
0.5917517095361369
доброе утро
1.0
что делаешь
1.0


### По наиболее схожему вопросу найти ответ

In [22]:
df = pd.read_excel('questions.xlsx')
df.head()

,question,answer
0,1. Что такое программирование?,Программирование – это процесс создания компью...
1,2. Как работает программный код?,"Код программы содержит инструкции, которые явл..."
2,3. Что такое отладка?,Отладка – это процесс поиска и устранения ошиб...
3,"4. Назовите типы ошибок, которые могут возникн...",Синтаксические ошибки.\nRuntime errors или оши...
4,5. Расскажите о синтаксических ошибках,"Синтаксическая ошибка возникает, когда в коде ..."


In [24]:
list(df['question'])

['1. Что такое программирование?',
 '2. Как работает программный код?',
 '3. Что такое отладка?',
 '4. Назовите типы ошибок, которые могут возникнуть в программе',
 '5. Расскажите о синтаксических ошибках',
 '6. Расскажите об ошибке времени выполнения',
 '7. Расскажите о логических ошибках',
 '8. Что такое блок-схема?',
 '9. Что такое алгоритм?',
 '10. Что по вашему значит «Поддерживать и обновлять программу»?',
 '11. Что такое переменные?',
 '12. Что такое зарезервированные слова в\xa0программировании?',
 '13. Что такое цикл?',
 '14. Назовите разные виды циклов',
 '15. Чем отличаются for и while?',
 '16. Что такое вложенный цикл?',
 '17. Что такое документация?',
 '18. Что делает компилятор?',
 '19. Что такое двоичный код?',
 '20. Что такое константа?',
 '21. Что такое операторы?',
 '22. Что такое массив?',
 '23. Что такое подпрограмма?',
 '24. Когда применяют арифметические операторы?',
 '25. Реляционные операторы',
 '26. Для чего нужны логические операторы?',
 '27. Зачем нужны опера